In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
train_p_d = pd.read_csv('/content/train_data_clinical_patient.csv')
train_rna = pd.read_csv('/content/train_data_rna-seq.csv')
test_rna = pd.read_csv('/content/test_data_rna-seq_v1.csv')

In [ ]:
#train 전처리부분
train_rna = train_rna.dropna(axis=0)
train_rna = train_rna.transpose()
train_rna = train_rna[2:]
train_rna = train_rna.rename(lambda label:label[:-3])
train_rna_id = list(train_rna.index)

In [ ]:
#test 전처리부분
test_rna = test_rna.dropna(axis=0)
test_rna = test_rna.transpose()
test_rna = test_rna.rename(lambda label:label[8:])
test_rna['sort'] = test_rna.index
test_rna = test_rna.iloc[2:]
test_rna = test_rna.astype(np.float64)
test_rna = test_rna.sort_values(by = 'sort', ascending=True)
test_rna = test_rna.drop(columns='sort')

In [ ]:
#train_data_clinical_patient에서 환자ID 추출
train_p_d = train_p_d.rename(index=train_p_d["PATIENT_ID"])
p_id = list(train_p_d["PATIENT_ID"].values)

In [ ]:
rna_re = []
for i in train_rna_id:
    if i not in p_id:
        rna_re.append(i)
train_rna = train_rna.drop(index=rna_re)

In [ ]:
patient_re = []
for i in p_id:
    if i not in train_rna_id:
        patient_re.append(i)
train_p_d = train_p_d.drop(index=patient_re)

In [ ]:
!pip install iteration_utilities

In [ ]:
from iteration_utilities import duplicates

In [ ]:
remove_du = list(duplicates(list(train_rna.index)))
train_rna = train_rna.drop(index=remove_du)
du = list(duplicates(list(train_p_d.index)))
train_p_d = train_p_d.drop(index=du)

In [ ]:
p_id = list(train_rna.index)
train_rna_id = list(train_p_d.index)
re = []
for i in train_rna_id:
    if i not in p_id:
        train_p_d = train_p_d.drop(index=i)

In [ ]:
cancer_name = train_p_d['CANCER_TYPE_ACRONYM']
X = train_rna.iloc[:,:300]
y = cancer_name
test_rna = test_rna.iloc[:,:300]
X.shape

(2603, 200)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

encoder.fit(y)
y_encoded = encoder.transform(y)

for label in np.unique(y):
    if label not in encoder.classes_: 
        encoder.classes_ = np.append(encoder.classes_, label)
y_encoded = encoder.transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size= 0.3)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
# random forest 모델 학습
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=700, max_depth = 20)

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# random forest 정확도 
score = rf.score(X_test, y_test)
print(score)

In [ ]:
# random forest 모델 예측
pred = rf.predict(test_rna)

In [ ]:
pred = encoder.inverse_transform(pred)

In [ ]:
pred = pd.DataFrame(pred)
submission_file = pd.read_csv('/content/sample_submission.csv')
submission_file = submission_file.drop(['expected'],axis=1)
submission_file['expected']= pred
submission_file.to_csv("/content/new_submission_file.csv",index = False)